In [2]:
import pandas as pd
fake_real_df = pd.read_csv('./fake_or_real_news.csv')

In [3]:
fake_real_df_clean = fake_real_df[['title','text','label']]

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(fake_real_df_clean, fake_real_df_clean["label"]):
    strat_train_set = fake_real_df_clean.loc[train_index]
    strat_test_set = fake_real_df_clean.loc[test_index]

In [5]:
strat_fake_or_real_train = strat_train_set.copy()
strat_fake_or_real_test  = strat_test_set.copy()

In [6]:
X_train = strat_fake_or_real_train.copy().drop('label', 1)
y_train = strat_fake_or_real_train.copy()['label']
X_test  = strat_fake_or_real_test.copy().drop('label', 1)
y_test  = strat_fake_or_real_test.copy()['label']

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer

def preprocessing(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop]
    tokens = [word for word in tokens if len(word) >= 3]
    tokens = [word.lower() for word in tokens]
    lcStem = LancasterStemmer()
    tokens = [lcStem.stem(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [8]:
X_train_text_list  = list(X_train['text'])
X_train_title_list = list(X_train['title'])
X_test_text_list   = list(X_test['text'])
X_test_title_list  = list(X_test['title'])

In [9]:
X_train_text_clean = [preprocessing(text) for text in X_train_text_list]
X_train_title_clean = [preprocessing(text) for text in X_train_title_list]
X_test_text_clean = [preprocessing(text) for text in X_test_text_list]
X_test_title_clean = [preprocessing(text) for text in X_test_title_list]

In [11]:
y_train_binary = [x=='REAL' for x in list(y_train)]
y_test_binary = [x=='REAL' for x in list(y_test)]

In [12]:
'''
The code above is all cleaning.
I separated the two features Text and title. 
You can join them into a df
if you want to use multiple features analysis.
Below is the actual machine learning that I have done
so far.
'''
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,2), stop_words='english', strip_accents='unicode',norm=u'l2')

X_train_tfidf = vectorizer.fit_transform(X_train_text_clean, y_train_binary)

In [13]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train_binary)

In [14]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf, X_train_tfidf, y_train_binary, cv=5, scoring="accuracy")

array([ 0.85024631,  0.82840237,  0.83810464,  0.84698914,  0.86080948])

In [15]:
'''
It is now up to you to use the data as you wish or change
anything.
I am still going to work on improving the current model.
'''

'\nIt is now up to you to use the data as you wish or change\nanything.\nI am still going to work on improving the current model.\n'